In [1]:
import numpy as np
import pandas as pd
import time
from datetime import timedelta
from IPython.display import clear_output
import ipywidgets as widgets

import sys
sys.path.append("../..")
from utils import utils, postprocessing_utils as post
from fine_tuning import fine_tuning

In [2]:
import logging
logging.getLogger("transformers").setLevel(logging.ERROR)

In [3]:
data_path = "../../data/sentiment/"
short_model_name = input("Short model name: ")
experiment = input("Experiment: ")
task = "sentiment"
print("\n")
checkpoints_path = "E:/TFM_CCIL/checkpoints_score_limit/"

# Target score
M = post.Metrics(results_dir="../../results/",
                 experiment="acl",
                 short_model_name=short_model_name,
                 task=task,
                 metric="Macro_F1")
df = M.within_score()
score_limit = np.min(df["Within-Score"])
print("Score limit:", score_limit, "\n")

batches_per_eval = 128

# Model parameters
max_length = 256
batch_size = 8
learning_rate = 2e-5
epochs = 30
num_labels = 2
use_class_weights = True


training_lang = fine_tuning.get_global_training_state(data_path, short_model_name, experiment, checkpoints_path)

Short model name: xlm-roberta
Experiment: acl


Score limit: 0.6237154150197628 

Training language:   Japanese 

Already trained:     Spanish  Norwegian  Chinese     Thai       Indonesian  Basque  Turkish  Hebrew    Maltese
German               Slovak   Greek      Vietnamese  Cantonese  Finnish     Korean  Arabic   Algerian

Not yet trained:   

Cannot train:      



In [4]:
scores = widgets.Output()
scores

Output()

In [5]:
start = time.time()

while training_lang is not None:
    trainer = fine_tuning.LimitTrainer(score_limit, training_lang, data_path, task, short_model_name, 
                                       use_class_weights=use_class_weights)
    trainer.build_model(max_length, batch_size, learning_rate, epochs, num_labels, eval_batch_size=64)
    trainer.setup_checkpoint(checkpoints_path)
    trainer.prepare_data()
    
    dev_score = trainer.train(batches_per_eval=batches_per_eval, auto_adjust=True, save_freq=1)
    with scores:
        print("{}: {}".format(utils.code_to_name[training_lang], dev_score))
    trainer.model = None # Free memory
    time.sleep(10)
        
    clear_output()
    total_elapsed = time.time() - start
    print("Total time elapsed:", timedelta(seconds=np.round(total_elapsed)), "\n")
    training_lang = fine_tuning.get_global_training_state(data_path, short_model_name, experiment, checkpoints_path)

Successfully built tf-xlm-roberta-base
Final file: E:/TFM_CCIL/checkpoints_score_limit/ja/tf-xlm-roberta-base_sentiment.hdf5
Temp weights file: E:/TFM_CCIL/checkpoints_score_limit/ja/tf-xlm-roberta-base_temp.hdf5
Calculating class weights
{0: 1.352290193667384, 1: 0.7933273219116321}
Epoch 0
-------



36/36 [==============================] - 41s 1s/step
Dev Score: 0.5817795394809786
Saving temp weights...



36/36 [==============================] - 41s 1s/step
Dev Score: 0.4956225374310481
Saving temp weights...



36/36 [==============================] - 41s 1s/step
Dev Score: 0.37154041724703946
Saving temp weights...



36/36 [==============================] - 41s 1s/step
Dev Score: 0.39657562224679854
Saving temp weights...



36/36 [==============================] - 41s 1s/step
Dev Score: 0.39657562224679854
Saving temp weights...



36/36 [==============================] - 41s 1s/step
Dev Score: 0.40169940298623863
Saving temp weights...



36/36 [==============================]

KeyboardInterrupt: 